# 2-Categories
Categories, Functors, and Natural Transformations together make up the structure of a 2-category.

In [144]:
:ext KindSignatures
:ext MultiParamTypeClasses
:ext GADTs
:ext PolyKinds
:ext FunctionalDependencies
:ext RankNTypes
:ext TypeFamilies
:ext FlexibleContexts
:ext AllowAmbiguousTypes
:ext TypeOperators
:ext UndecidableInstances
:ext TypeApplications
:ext ScopedTypeVariables

import Data.Proxy
import Prelude hiding (Functor, id, (.))

class Category (m :: * -> * -> *) where
  id :: m a a
  (.) :: m b c -> m a b -> m a c
  
type Obj c a = Category c => c a a

class (Category (Dom fsym), Category (Cod fsym)) => Functor fsym where
  type Dom fsym :: * -> * -> *
  type Cod fsym :: * -> * -> *
  type MObj fsym a :: *
  fmap :: Proxy fsym -> Dom fsym i j -> Cod fsym (MObj fsym i) (MObj fsym j)

data f :.: g where
  (:.:) :: (Functor f, Functor g, Cod g ~ Dom f) => Proxy f -> Proxy g -> f :.: g
  
instance (Functor f, Functor g, Cod g ~ Dom f) => Functor (f :.: g) where
  type Dom (f :.: g) = Dom g
  type Cod (f :.: g) = Cod f
  type MObj (f :.: g) x = MObj f (MObj g x)
  fmap _ x = fmap (Proxy @f) (fmap (Proxy @g) x)
  
type Component f g x = Cod f (MObj f x) (MObj g x)

data NT c d f g where
  NT :: (Functor f, Functor g, c ~ Dom f, c ~ Dom g, d ~ Cod f, d ~ Cod g)
     => Proxy f -> Proxy g -> (forall x. Obj c x -> (MObj f x -> Component f g x))
     -> NT c d f g
     
--data f :..: g where
--  (:..:) :: (c ~ Dom f, c ~ Dom g, d ~ Cod f, d ~ Cod g) => NT c d f g -> f :..: g
  
instance Category (NT c d) where
  id = NT Proxy Proxy id
--  (.) = _2

--instance Category (:..:)
  
--type f :~> g = forall c d. (c ~ Dom f, c ~ Dom g, d ~ Cod f, d ~ Cod g)
--            => NT c d f g
     
-- I want to have category instances for
-- Endofunctors
-- Functors with same Dom, Cod - NT as morphism
-- Vertically composing NTs
-- Horizontally composing NTs

--type family Comp (f :: b -> c) (g :: a -> b) (x :: a) :: c where
-- Comp f g x = f (g x)

{-
data NT (c :: x -> x -> *) (d :: y -> y -> *) (m :: x -> y) (m' :: x -> y) where
  NT :: (CFunctor c d m, CFunctor c d m')
     => (d (m a) (m b)
     -> d (m' a) (m' b))
     -> NT c d m m'
  
  VComp :: NT c d f g -> NT c d g h -> NT c d f h
  
  HComp :: ( Category x c, Category y d, Category z e 
           , CFunctor c d f, CFunctor c d g, CFunctor d e h, CFunctor d e i)
        => NT c d (f :: x -> y) (g :: x -> y)
        -> NT d e (h :: y -> z) (i :: y -> z)
        -> NT c e (hf :: x -> z) (ig :: x -> z) -}


<interactive>:23:8: error:
    • Couldn't match type ‘c’ with ‘Dom a’ arising from a use of ‘NT’
      ‘c’ is a rigid type variable bound by
        the instance declaration
        at <interactive>:22:10-26
    • In the expression: NT Proxy Proxy id
      In an equation for ‘id’: id = NT Proxy Proxy id
      In the instance declaration for ‘Category (NT c d)’
    • Relevant bindings include id :: NT c d a a (bound at <interactive>:23:3)
<interactive>:23:23: error:
    • Couldn't match kind ‘*’ with ‘Constraint’
      When matching types
        MObj f10 x :: *
        Category c :: Constraint
      Expected type: Obj c x -> MObj f10 x -> Component a a x
        Actual type: (MObj f10 x -> Cod a (MObj a x) (MObj a x)) -> MObj f10 x -> Cod a (MObj a x) (MObj a x)
    • In the third argument of ‘NT’, namely ‘id’
      In the expression: NT Proxy Proxy id
      In an equation for ‘id’: id = NT Proxy Proxy id
    • Relevant bindings include id :: NT c d a a (bound at <interactive>:23:3)